# ML Pipeline

> This pipeline uses each match's outcome to learn which player shall make it to the round of 16

- The model trains on data recorded in the prior 3 years of the testing year
- Post training, it predicts the first round of the tournament in two ways:
    - Walkovers (given manually) for players who compete with players who don't have statistics
    - model's predictions for matches where both players have statistics
- Testing data is reconstructed to predict winners of the second round
- Only relies on model for prediction of winners of second round
- Testing data is reconstructed to predict winners of the third round. This is also the last loop of testing.
- Only relies on model for prediction of winners of third round
- Compare with the actual results and winners of third round

# Import Libraries

In [41]:
import pandas as pd
from difflib import get_close_matches
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


# Inputs

In [42]:
TRAIN_ON = [2015, 2020]
TEST_ON = 2020
mod1 = MLPClassifier(activation='logistic', solver='adam', 
                      learning_rate='adaptive', max_iter=10000, warm_start=True,
                      hidden_layer_sizes=(128,50,2))
mod2 = SVC(kernel='linear',C=0.001, max_iter=1000000)
mod3 = GradientBoostingClassifier(n_estimators=10000, learning_rate=0.00001, criterion='squared_error', loss='exponential')
MODEL1 = mod1
MODEL2 = mod2
MODEL3 = mod3

# Utils

In [44]:
stats = ['aces_per_match', 'double_faults_per_match', 'break_points_faced_per_match',
         'break_points_opportunities_per_match', 'height (cm)',
         'matches_played', '1st_serve',
         'return_games_won', '1st_serve_points_won', '2nd_serve_points_won',
         'service_games_won', '1st_serve_return_points_won',
         '2nd_serve_return_points_won', 'pressure_rating']

# DO NOT CHANGE THIS: It conatins the order of matches so that the match seeds are intact
actual_seed = [48, 43, 34, 42, 35, 53, 57, 40, 52, 49, 63, 54, 55, 59, 46,
               58, 39, 60, 41, 33, 61, 56, 50, 62, 51, 44, 38, 37, 45, 36,
               32, 47, 8, 10, 9, 15, 21, 25, 6, 7, 14, 24, 20, 18, 13, 11,
               26, 19, 16, 4, 3, 2, 28, 5, 12, 1, 17, 27, 30, 22, 29, 0,
               23, 31,]
# 2018 seedings
actual_seed = [6, 13, 24, 8, 12, 25, 11, 29, 19, 7, 28, 21, 22, 16,
               4, 15, 3, 30, 17, 9, 31, 26, 18, 2, 1, 5, 27, 20,
               0, 10, 14, 23, 59, 34, 57, 36, 49, 35, 58, 37,
               61, 56, 47, 48, 46, 39, 43, 50, 33, 40, 38, 42,
               51, 62, 45, 41, 60, 54, 53, 44, 52, 63, 32, 55]

actual_seed = [
    32, 34, 60, 61, 36, 35, 37, 40, 47, 43, 48,
    49, 63, 53, 57, 44, 62, 45, 52, 56, 55, 39, 41, 59, 58,
    51, 42, 7, 15, 50, 33, 23, 11, 46, 38, 10, 3, 0,
    6, 54, 28, 29, 14, 22, 19, 27, 2, 26, 25, 16, 8, 4, 18,
    12, 24, 1, 21, 31, 20, 17, 13, 9, 30, 5,
]


def change_name(name: str) -> str:
    """change order of name from first last to last first

    Args:
        name (str): name of player as first last

    Returns:
        str: last first name
    """
    split_names = name.split()
    f_name = split_names.pop(0)
    split_names.append(f_name)
    final_name = " ".join(split_names)
    return final_name


def rank_gr(rank: float, group_lim: int = 15):
    ranges = list(range(0, 100, group_lim))
    if rank not in list(range(1, 1000)):
        return len(ranges)

    for idx, num in enumerate(ranges):
        if num - rank >= 0:
            return idx
    return len(ranges)


def stat_df(year: int) -> pd.DataFrame:
    """returns stats of players participating in the next given year's AO

    Args:
        year (int): year of stats needed

    Returns:
        pd.DataFrame: stats of players participating in the next given year's AO
    """
    df = pd.read_csv(f'./data/aus-open-player-stats-{year}.csv')
    # df['rank_group'] = df['rank_x'].apply(lambda x: rank_gr(x))
    # grp_rank_pr = df.groupby('rank_group').agg({'pressure_rating':'mean'}).reset_index()

    # grp_rank_pr.set_index('rank_group', inplace=True)
    # df.set_index('rank_group', inplace=True)

    # df['pressure_rating'].fillna(grp_rank_pr['pressure_rating'], inplace=True)

    # df.reset_index(drop=True, inplace=True)

    df['pressure_rating'].fillna(0, inplace=True)

    df.loc[df.aces == 0, 'aces'] = -np.inf

    per_match_cols = ['aces', 'double_faults',
                      'break_points_opportunities', 'break_points_faced']
    for col in per_match_cols:
        df[f'{col}_per_match'] = df[col] / df.matches_played

    percent_cols = ["1st_serve", "1st_serve_points_won", "2nd_serve_points_won", "break_points_saved",
                    "service_games_won", "1st_serve_return_points_won", "2nd_serve_return_points_won",
                    "break_points_converted", "return_games_won", 'total_points_won', 'return_points_won']
    for col in percent_cols:
        df[col] = df[col].apply(lambda x: round(float(x.strip('%')) / 100, 2))

    per_match_cols = [f'{x}_per_match' for x in per_match_cols]
    per_match_cols.extend(percent_cols)
    per_match_cols.extend(['name', 'year', 'height (cm)',
                          'matches_played', 'pressure_rating'])
    return df[per_match_cols]


In [45]:
def year_df(year: int) -> pd.DataFrame:
    """provides training data for the given year

    Args:
        year (int): year of the tournament

    Returns:
        pd.DataFrame: training data for the given year with the player's stats in the previous year
    """
    df = pd.read_csv(f'./data/m{year}.csv')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True).dt.year
    df = df[['Round', 'Winner', 'Loser', 'WRank', 'LRank', 'Date']]

    even = df.iloc[::2,].copy(deep=True)
    odd = df.iloc[1::2,].copy(deep=True)
    even.rename(columns={'Winner': 'player_0', 'Loser': 'player_1',
                'WRank': 'rank_0', 'LRank': 'rank_1'}, inplace=True)
    odd.rename(columns={'Winner': 'player_1', 'Loser': 'player_0',
                'WRank': 'rank_1', 'LRank': 'rank_0'}, inplace=True)
    even['winner'] = 0
    odd['winner'] = 1


    final_df = pd.concat([even, odd]).sort_index()
    final_df['rank'] = final_df['rank_0'] - final_df['rank_1']
    # final_df.drop(columns=['rank'], inplace=True)

    names_to_lookup = set(final_df.player_0)
    names_to_lookup.update(set(final_df.player_1))

    df2 = stat_df(year - 1)

    df2['new_name'] = df2['name'].apply(change_name)
    df2['other_name'] = df2['new_name'].apply(
        lambda x: get_close_matches(x, names_to_lookup, 1, cutoff=0.5)[0])

    df2 = df2.drop(columns=['new_name'])

    final_df[stats] = final_df.apply(lambda x: df2.loc[df2.other_name == x.player_0,stats].copy(deep=True).iloc[0,] - df2.loc[df2.other_name == x.player_1,stats].copy(deep=True).iloc[0,], axis=1)
    return final_df


# Training data

In [46]:
final_df = pd.DataFrame()
for year in range(TRAIN_ON[0],TRAIN_ON[1]):
    final_df = pd.concat([final_df, year_df(year)])
final_df

,Round,player_0,player_1,rank_0,rank_1,Date,winner,rank,aces_per_match,double_faults_per_match,...,height (cm),matches_played,1st_serve,return_games_won,1st_serve_points_won,2nd_serve_points_won,service_games_won,1st_serve_return_points_won,2nd_serve_return_points_won,pressure_rating
0,1st Round,Berankis R.,Sijsling I.,85.0,84.0,2015,0,1.0,-5.551724,-1.706897,...,-16.0,32.0,0.00,-0.03,-0.05,0.00,-0.04,0.02,-0.08,39.4
1,1st Round,Brown D.,Dimitrov G.,90.0,11.0,2015,1,79.0,-1.868132,-0.817582,...,5.0,-22.0,0.04,-0.06,-0.04,-0.03,-0.02,-0.04,0.02,-68.1
2,1st Round,Anderson K.,Schwartzman D.,15.0,59.0,2015,0,-44.0,9.104343,2.211335,...,33.0,27.0,0.16,0.03,0.15,0.17,0.44,0.03,0.10,152.1
3,1st Round,Coric B.,Chardy J.,91.0,31.0,2015,1,60.0,-7.867821,-4.334382,...,0.0,10.0,-0.05,0.05,-0.02,0.02,0.01,0.02,0.02,-35.9
4,1st Round,Lacko L.,Gonzalez M.,99.0,98.0,2015,0,1.0,2.426471,0.558824,...,15.0,26.0,-0.02,0.09,0.03,-0.07,-0.02,0.07,0.04,162.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Quarterfinals,Pouille L.,Raonic M.,31.0,17.0,2019,0,14.0,-4.223587,-0.176904,...,-11.0,-4.0,-0.05,0.03,-0.08,-0.05,-0.06,-0.01,0.04,34.2
123,Quarterfinals,Nishikori K.,Djokovic N.,9.0,1.0,2019,1,8.0,-1.258242,-0.474359,...,-10.0,10.0,-0.04,-0.05,-0.03,-0.02,-0.06,-0.03,-0.04,-1.9
124,Semifinals,Nadal R.,Tsitsipas S.,2.0,15.0,2019,0,-13.0,-2.003857,0.095468,...,-8.0,-44.0,0.09,0.12,-0.05,0.01,-0.01,0.07,0.07,15.0
125,Semifinals,Pouille L.,Djokovic N.,31.0,1.0,2019,1,30.0,4.038961,1.196970,...,-3.0,-9.0,-0.08,-0.11,0.01,-0.08,-0.05,-0.08,-0.07,-19.9


# Preprocessing

## Drop NA Values

In [47]:
final_df = final_df.replace([np.inf, -np.inf], np.nan).dropna()

## One Hot Encoding

In [48]:
final_df = pd.get_dummies(final_df, columns=['Round'])
final_df

,player_0,player_1,rank_0,rank_1,Date,winner,rank,aces_per_match,double_faults_per_match,break_points_faced_per_match,...,1st_serve_return_points_won,2nd_serve_return_points_won,pressure_rating,Round_1st Round,Round_2nd Round,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Semifinals,Round_The Final
0,Berankis R.,Sijsling I.,85.0,84.0,2015,0,1.0,-5.551724,-1.706897,-3.454907,...,0.02,-0.08,39.4,1,0,0,0,0,0,0
1,Brown D.,Dimitrov G.,90.0,11.0,2015,1,79.0,-1.868132,-0.817582,-1.215385,...,-0.04,0.02,-68.1,1,0,0,0,0,0,0
2,Anderson K.,Schwartzman D.,15.0,59.0,2015,0,-44.0,9.104343,2.211335,4.269068,...,0.03,0.10,152.1,1,0,0,0,0,0,0
3,Coric B.,Chardy J.,91.0,31.0,2015,1,60.0,-7.867821,-4.334382,-5.548387,...,0.02,0.02,-35.9,1,0,0,0,0,0,0
4,Lacko L.,Gonzalez M.,99.0,98.0,2015,0,1.0,2.426471,0.558824,1.632353,...,0.07,0.04,162.8,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Pouille L.,Raonic M.,31.0,17.0,2019,0,14.0,-4.223587,-0.176904,2.386568,...,-0.01,0.04,34.2,0,0,0,0,1,0,0
123,Nishikori K.,Djokovic N.,9.0,1.0,2019,1,8.0,-1.258242,-0.474359,0.723443,...,-0.03,-0.04,-1.9,0,0,0,0,1,0,0
124,Nadal R.,Tsitsipas S.,2.0,15.0,2019,0,-13.0,-2.003857,0.095468,3.571842,...,0.07,0.07,15.0,0,0,0,0,0,1,0
125,Pouille L.,Djokovic N.,31.0,1.0,2019,1,30.0,4.038961,1.196970,1.837662,...,-0.08,-0.07,-19.9,0,0,0,0,0,1,0


# Modeling

In [49]:
X_train = final_df.drop(columns=['winner','player_0','player_1','rank_0','rank_1'])
y_train = final_df['winner']

In [50]:

MODEL1.fit(X_train, y_train)
MODEL2.fit(X_train, y_train)
MODEL3.fit(X_train, y_train)

GradientBoostingClassifier(criterion='squared_error', learning_rate=1e-05,
                           loss='exponential', n_estimators=10000)

# Predicting

In [51]:
# We get data of players participating in 2019, 
# and their stats in 2018
main_df = year_df(TEST_ON)
main_df = pd.get_dummies(main_df, columns=['Round'])
cols_to_have = main_df.columns
main_df = main_df.loc[:63,].copy(deep=True)

In [22]:
main_df.to_csv('test2018.csv',index=False)

In [52]:
main_df.index = actual_seed
main_df.sort_index(inplace=True)
test = main_df.replace([np.inf,-np.inf],np.nan)
test = test.dropna()

# # test.drop(columns=['player_0','player_1','rank_0','rank_1'], inplace=True)
X_test = test.drop(columns=['winner','player_0','player_1','rank_0','rank_1'])
y_test = test['winner']

First Round (Round of 64) prediction accuracy

In [55]:
MODEL2.score(X_test, y_test)

0.7884615384615384

# First Round Predictions

In [56]:
round_64 = MODEL3.predict(X_test)

In [57]:
test = test.reset_index(drop=True)
test['prediction'] = round_64

In [58]:
# this dataframe helps in including the manual and model predictions
middle_man = main_df.join(test[['player_0', 'player_1',
                                'rank_0', 'rank_1', 'prediction']]
                          .set_index(['player_0', 'player_1',
                                      'rank_0', 'rank_1']),
                          on=['player_0', 'player_1', 'rank_0', 'rank_1'])


## Force a certian prediction

In [59]:
middle_man.loc[(middle_man.aces_per_match == np.inf) |
               (middle_man.aces_per_match == -np.inf), 'prediction'] = middle_man.loc[(middle_man.aces_per_match == np.inf) |
                                                                                      (middle_man.aces_per_match == -np.inf),].apply(lambda x: 1 if x.aces_per_match == -np.inf else 0, axis=1)


In [60]:
middle_man['w_name'] = middle_man.apply(lambda x: x.player_0 if x.prediction == 0 else x.player_1, axis=1)
middle_man['w_rank'] = middle_man.apply(lambda x: x.rank_0 if x.prediction == 0 else x.rank_1, axis=1)

# 2nd Round predictions (Round of 32)

## Preprocessing

In [61]:
round_32 = middle_man[['w_name','w_rank']].copy(deep=True)

In [62]:
r_32_even = round_32.iloc[::2,].copy(deep=True).reset_index(drop=True)
r_32_odd = round_32.iloc[1::2,].copy(deep=True).reset_index(drop=True)

In [63]:
r_32_even.columns = ['player_0','rank_0']
r_32_odd.columns = ['player_1','rank_1']

In [64]:
r_32_merge = pd.concat([r_32_even,r_32_odd], axis=1)
r_32_merge['Round'] = '2nd Round'

In [65]:
df2 = stat_df(TEST_ON - 1)

names_to_lookup = set(middle_man.player_0)
names_to_lookup.update(middle_man.player_1)

df2['new_name'] = df2['name'].apply(change_name)
df2['other_name'] = df2['new_name'].apply(
    lambda x: get_close_matches(x, names_to_lookup, 1, cutoff=0.5)[0])

df2 = df2.drop(columns=['new_name'])

In [66]:
r_32_merge[stats] = r_32_merge.apply(lambda x: df2.loc[df2.other_name == x.player_0,stats].copy(deep=True).iloc[0,] - df2.loc[df2.other_name == x.player_1,stats].copy(deep=True).iloc[0,], axis=1)

In [67]:
r_32_merge['rank'] = r_32_merge['rank_0'] - r_32_merge['rank_1']
r_32_merge = pd.get_dummies(r_32_merge, columns=['Round'])

In [68]:
r_32_merge[['Round_1st Round','Round_3rd Round', 'Round_4th Round', 'Round_Quarterfinals','Round_Semifinals','Round_The Final']] = 0
r_32_merge['Date'] = pd.to_datetime('2019')
r_32_merge['Date'] = pd.to_datetime(r_32_merge['Date'], yearfirst=True).dt.year

## Predicting

In [70]:
X_test = r_32_merge.drop(columns=['player_0', 'player_1', 'rank_0', 'rank_1'])
r_32_pred = MODEL2.predict(X_test[X_train.columns])
r_32_merge['prediction'] = r_32_pred

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [71]:
r_32_merge

,player_0,rank_0,player_1,rank_1,aces_per_match,double_faults_per_match,break_points_faced_per_match,break_points_opportunities_per_match,height (cm),matches_played,...,pressure_rating,rank,Round_2nd Round,Round_1st Round,Round_3rd Round,Round_4th Round,Round_Quarterfinals,Round_Semifinals,Round_The Final,Date
0,Nadal R.,1.0,Sousa J.,59.0,5.704412,2.622059,1.872059,11.502941,0.0,-23.0,...,42.3,-58.0,1,0,0,0,0,0,0,2019
1,Eubanks C.,221.0,Carreno Busta P.,30.0,-4.012366,-1.303226,-5.849462,-7.314516,13.0,29.0,...,-29.4,191.0,1,0,0,0,0,0,0,2019
2,Kyrgios N.,26.0,Simon G.,61.0,11.824760,0.897123,-1.802092,-2.431561,10.0,-6.0,...,39.4,-35.0,1,0,0,0,0,0,0,2019
3,Ymer M.,78.0,Khachanov K.,17.0,-4.567751,-1.676152,-2.808943,-3.219512,-15.0,-13.0,...,-10.1,61.0,1,0,0,0,0,0,0,2019
4,Monfils G.,10.0,Karlovic I.,124.0,-3.937500,3.281250,6.696429,10.839286,-18.0,-4.0,...,30.9,-114.0,1,0,0,0,0,0,0,2019
5,Bedene A.,55.0,Auger-Aliassime F.,22.0,5.764706,-0.323529,1.323529,1.529412,-10.0,-17.0,...,-17.7,33.0,1,0,0,0,0,0,0,2019
6,Fritz T.,34.0,Anderson K.,122.0,3.619296,0.926988,2.281617,1.927640,-7.0,-7.0,...,-4.8,-88.0,1,0,0,0,0,0,0,2019
7,Ramos-Vinolas A.,42.0,Thiem D.,5.0,-2.253205,-0.358974,0.560897,-1.352564,3.0,-15.0,...,-83.9,37.0,1,0,0,0,0,0,0,2019
8,Medvedev D.,4.0,Koepfer D.,89.0,7.752625,2.342125,3.327473,5.897192,18.0,2.0,...,19.1,-85.0,1,0,0,0,0,0,0,2019
9,Munar J.,90.0,Tsonga J.W.,33.0,-29.833333,-5.625000,-10.885417,-11.927083,-5.0,20.0,...,-96.7,57.0,1,0,0,0,0,0,0,2019


In [39]:
r_32_merge['w_name'] = r_32_merge.apply(lambda x: x.player_0 if x.prediction == 0 else x.player_1, axis=1)
r_32_merge['w_rank'] = r_32_merge.apply(lambda x: x.rank_0 if x.prediction == 0 else x.rank_1, axis=1)

AttributeError: 'Series' object has no attribute 'prediction'

# 3rd Round predictions (Round of 16)

## Preprocessing

In [118]:
r_16 = r_32_merge[['w_name','w_rank']].copy(deep=True)

In [119]:
r_16_even = r_16.iloc[::2,].copy(deep=True).reset_index(drop=True)
r_16_odd = r_16.iloc[1::2,].copy(deep=True).reset_index(drop=True)

In [120]:
r_16_even.columns = ['player_0','rank_0']
r_16_odd.columns = ['player_1','rank_1']

In [121]:
r_16_merge = pd.concat([r_16_even,r_16_odd], axis=1)
r_16_merge['Round'] = '3rd Round'

In [122]:
r_16_merge[stats] = r_16_merge.apply(lambda x: df2.loc[df2.other_name == x.player_0,stats].copy(deep=True).iloc[0,] - df2.loc[df2.other_name == x.player_1,stats].copy(deep=True).iloc[0,], axis=1)

In [123]:
r_16_merge['rank'] = r_16_merge['rank_0'] - r_16_merge['rank_1']
r_16_merge = pd.get_dummies(r_16_merge, columns=['Round'])

In [124]:
r_16_merge[['Round_1st Round','Round_2nd Round', 'Round_4th Round', 'Round_Quarterfinals','Round_Semifinals','Round_The Final']] = 0
r_16_merge['Date'] = pd.to_datetime('2019')
r_16_merge['Date'] = pd.to_datetime(r_16_merge['Date'], yearfirst=True).dt.year

In [125]:
X_test = r_16_merge.drop(columns=['player_0','player_1','rank_0','rank_1'])
r_16_pred = MODEL3.predict(X_test[X_train.columns])

In [126]:
r_16_merge['prediction']= r_16_pred

# Final Contenders

In [127]:
r_16_merge['w_name'] = r_16_merge.apply(lambda x: x.player_0 if x.prediction == 0 else x.player_1, axis=1)
r_16_merge['w_rank'] = r_16_merge.apply(lambda x: x.rank_0 if x.prediction == 0 else x.rank_1, axis=1)
r_16_merge[['player_0','player_1','rank_0','rank_1','w_name','w_rank']]

,player_0,player_1,rank_0,rank_1,w_name,w_rank
0,Nadal R.,Dzumhur D.,1,30,Nadal R.,1
1,Schwartzman D.,Ebden M.,26,78,Schwartzman D.,26
2,Carreno Busta P.,Muller G.,11,28,Muller G.,28
3,Harrison R.,Cilic M.,45,6,Cilic M.,6
4,Dimitrov G.,Rublev A.,3,32,Dimitrov G.,3
5,Kyrgios N.,Tsonga J.W.,17,15,Kyrgios N.,17
6,Anderson K.,Pouille L.,12,18,Anderson K.,12
7,Seppi A.,Sock J.,76,9,Sock J.,9
8,Thiem D.,Mannarino A.,5,27,Thiem D.,5
9,Bautista Agut R.,Wawrinka S.,21,8,Wawrinka S.,8


In [128]:
test_df = pd.read_csv(f'./data/m{TEST_ON}.csv')
winners = test_df.loc[test_df.Round == '3rd Round','Winner']

conf_matrix = pd.DataFrame(columns=['name','actual','pred'])
names = set(test_df.Loser)
names.update(test_df.Winner)
conf_matrix['name'] = pd.Series(list(names))
conf_matrix['actual'] = 0
conf_matrix['pred'] = 0
conf_matrix.loc[conf_matrix['name'].isin(winners),'actual'] = 1
pred_winners = r_16_merge['w_name']
conf_matrix.loc[conf_matrix['name'].isin(pred_winners),'pred'] = 1
conf_matrix

,name,actual,pred
0,Zeballos H.,0,0
1,Kicker N.,0,0
2,Kwon S.W.,0,0
3,Thiem D.,1,1
4,Anderson K.,0,1
...,...,...,...
123,Sonego L.,0,0
124,De Minaur A.,0,0
125,Chung H.,1,0
126,Dzumhur D.,0,0


In [129]:
confusion_matrix(conf_matrix['actual'],conf_matrix['pred'])

array([[104,   8],
       [  8,   8]])

In [130]:
acc = accuracy_score(conf_matrix['actual'],conf_matrix['pred'])
prec = precision_score(conf_matrix['actual'],conf_matrix['pred'])
rec = recall_score(conf_matrix['actual'],conf_matrix['pred'])
f1 = f1_score(conf_matrix['actual'],conf_matrix['pred'])
print(f'Accuracy: {acc}\nPrecision: {prec}\nRecall: {rec}\nF1: {f1}')

Accuracy: 0.875
Precision: 0.5
Recall: 0.5
F1: 0.5


In [131]:
test_df = pd.read_csv(f'./data/m{TEST_ON}.csv')
winners = test_df.loc[test_df.Round == '2nd Round','Winner']

conf_matrix = pd.DataFrame(columns=['name','actual','pred'])
names = set(test_df.Loser)
names.update(test_df.Winner)
conf_matrix['name'] = pd.Series(list(names))
conf_matrix['actual'] = 0
conf_matrix['pred'] = 0
conf_matrix.loc[conf_matrix['name'].isin(winners),'actual'] = 1
pred_winners = r_16['w_name']
conf_matrix.loc[conf_matrix['name'].isin(pred_winners),'pred'] = 1
conf_matrix

,name,actual,pred
0,Zeballos H.,0,0
1,Kicker N.,1,0
2,Kwon S.W.,0,0
3,Thiem D.,1,1
4,Anderson K.,0,1
...,...,...,...
123,Sonego L.,0,0
124,De Minaur A.,0,0
125,Chung H.,1,0
126,Dzumhur D.,1,1


In [132]:
confusion_matrix(conf_matrix['actual'],conf_matrix['pred'])

array([[86, 10],
       [10, 22]])

In [133]:
acc = accuracy_score(conf_matrix['actual'],conf_matrix['pred'])
prec = precision_score(conf_matrix['actual'],conf_matrix['pred'])
rec = recall_score(conf_matrix['actual'],conf_matrix['pred'])
f1 = f1_score(conf_matrix['actual'],conf_matrix['pred'])
print(f'Accuracy: {acc}\nPrecision: {prec}\nRecall: {rec}\nF1: {f1}')

Accuracy: 0.84375
Precision: 0.6875
Recall: 0.6875
F1: 0.6875


In [134]:
test_df = pd.read_csv(f'./data/m{TEST_ON}.csv')
winners = test_df.loc[test_df.Round == '1st Round','Winner']

conf_matrix = pd.DataFrame(columns=['name','actual','pred'])
names = set(test_df.Loser)
names.update(test_df.Winner)
conf_matrix['name'] = pd.Series(list(names))
conf_matrix['actual'] = 0
conf_matrix['pred'] = 0
conf_matrix.loc[conf_matrix['name'].isin(winners),'actual'] = 1
pred_winners = middle_man['w_name']
conf_matrix.loc[conf_matrix['name'].isin(pred_winners),'pred'] = 1
conf_matrix

,name,actual,pred
0,Zeballos H.,0,0
1,Kicker N.,1,0
2,Kwon S.W.,0,0
3,Thiem D.,1,1
4,Anderson K.,0,1
...,...,...,...
123,Sonego L.,1,0
124,De Minaur A.,0,0
125,Chung H.,1,0
126,Dzumhur D.,1,1


In [135]:
confusion_matrix(conf_matrix['actual'],conf_matrix['pred'])

array([[46, 18],
       [18, 46]])

In [98]:
acc = accuracy_score(conf_matrix['actual'],conf_matrix['pred'])
prec = precision_score(conf_matrix['actual'],conf_matrix['pred'])
rec = recall_score(conf_matrix['actual'],conf_matrix['pred'])
f1 = f1_score(conf_matrix['actual'],conf_matrix['pred'])
print(f'Accuracy: {acc}\nPrecision: {prec}\nRecall: {rec}\nF1: {f1}')

Accuracy: 0.703125
Precision: 0.703125
Recall: 0.703125
F1: 0.703125


In [ ]:
data = pd.read_csv()